Below code will generate Power meter reading data in 1 minute interval. While running, you will see message like below
"Generated 5 new readings to /Volumes/workspace/default/ext_volume/powerdata/raw_data//power_readings_20250806195341.json" 

In [0]:
# power_usage_simulator.py

import json
import time
import random
from datetime import datetime
from pyspark.sql.functions import lit
from pyspark.sql import SparkSession

# Initialize Spark Session (if not already in a Databricks notebook)
spark = SparkSession.builder.appName("PowerUsageSimulator").getOrCreate()

# Configuration
output_path = "/Volumes/workspace/default/ext_volume/powerdata/raw_data/"
num_customers = 50
min_kwh = 0.05
max_kwh = 5.5
interval_seconds = 10 # Generate a new batch of data every 10 seconds

def generate_power_reading():
    """Generates a single fake power reading for a specific customer and meter."""
    # Generate customer_id in the format CUST00001, CUST00002, etc.
    customer_id_int = random.randint(1, num_customers)
    customer_id = f"CUST{customer_id_int:05d}"
    
    # Generate metername in the format MTR00001, MTR00002, etc.
    metername_int = random.randint(1, num_customers)
    metername = f"MTR{metername_int:05d}"
    
    kwh_reading = round(random.uniform(min_kwh, max_kwh), 3)
    timestamp = datetime.now().isoformat()
    
    return {
        "customer_id": customer_id,
        "metername": metername,
        "kwh_reading": kwh_reading,
        "timestamp": timestamp,
        "raw_source": "simulated_smart_meter"
    }

def write_data_to_file():
    """Generates a batch of readings and writes them to a JSON file."""
    # Generate a random number of readings for each batch
    readings = [generate_power_reading() for _ in range(random.randint(5, 20))]
    
    # Create a temporary DataFrame
    df = spark.createDataFrame(readings)

    # Write the DataFrame to a JSON file in the specified path
    current_time = datetime.now().strftime("%Y%m%d%H%M%S")
    file_name = f"power_readings_{current_time}.json"
    full_path = f"{output_path}/{file_name}"
    
    df.write.json(full_path, mode="overwrite")
    print(f"Generated {len(readings)} new readings to {full_path}")
    
# Main loop to continuously generate data
while True:
    try:
        write_data_to_file()
        time.sleep(interval_seconds)
    except Exception as e:
        print(f"An error occurred: {e}")
        time.sleep(60) # Wait before retrying

Generated 18 new readings to /Volumes/workspace/default/ext_volume/powerdata/raw_data//power_readings_20250807022738.json
Generated 11 new readings to /Volumes/workspace/default/ext_volume/powerdata/raw_data//power_readings_20250807022756.json
Generated 10 new readings to /Volumes/workspace/default/ext_volume/powerdata/raw_data//power_readings_20250807022808.json
Generated 16 new readings to /Volumes/workspace/default/ext_volume/powerdata/raw_data//power_readings_20250807022820.json
Generated 6 new readings to /Volumes/workspace/default/ext_volume/powerdata/raw_data//power_readings_20250807022832.json
Generated 20 new readings to /Volumes/workspace/default/ext_volume/powerdata/raw_data//power_readings_20250807022844.json
Generated 10 new readings to /Volumes/workspace/default/ext_volume/powerdata/raw_data//power_readings_20250807022855.json
Generated 7 new readings to /Volumes/workspace/default/ext_volume/powerdata/raw_data//power_readings_20250807022907.json


com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:132)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:132)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:129)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:129)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:715)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:435)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:435)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
df=spark.read.json("/Volumes/workspace/default/ext_volume/powerdata/raw_data/power_readings_20250805201432.json")
display(df)

customer_id,kwh_reading,metername,raw_source,timestamp
CUST00016,1.733,MTR00019,simulated_smart_meter,2025-08-05T20:14:25.759470
CUST00044,3.494,MTR00046,simulated_smart_meter,2025-08-05T20:14:25.759483
CUST00046,3.283,MTR00001,simulated_smart_meter,2025-08-05T20:14:25.759492
CUST00033,0.115,MTR00028,simulated_smart_meter,2025-08-05T20:14:25.759497
CUST00045,0.188,MTR00008,simulated_smart_meter,2025-08-05T20:14:25.759501
CUST00042,3.893,MTR00023,simulated_smart_meter,2025-08-05T20:14:25.759505
CUST00046,3.005,MTR00016,simulated_smart_meter,2025-08-05T20:14:25.759508
CUST00043,4.587,MTR00041,simulated_smart_meter,2025-08-05T20:14:25.759515


In [0]:
%sql
select * from `power-catalog`.silverschema.silver_enriched_meter_readings

customer_id,first_name,last_name,city,state,kwh_reading,metername,raw_source,reading_timestamp,plan_id,plan_name,rate_per_kwh,effective_start_date,effective_end_date,processing_timestamp
CUST00001,Carla,Stewart,Cantuberg,MP,2.334,MTR00027,simulated_smart_meter,2025-08-06T19:54:53.803Z,PLAN001,Fixed Rate 12,0.1418,null,null,2025-08-07T02:50:58.086Z
CUST00002,Michael,Mullen,Riveraville,MA,0.556,MTR00014,simulated_smart_meter,2025-08-06T19:54:53.803Z,PLAN002,Variable Rate 6,0.1603,null,null,2025-08-07T02:50:58.086Z
CUST00003,Christina,Brooks,Millershire,MS,2.404,MTR00020,simulated_smart_meter,2025-08-07T02:28:44.038Z,PLAN003,Green Energy 24,0.2267,null,null,2025-08-07T02:50:58.086Z
CUST00004,Jesse,Perez,Suarezbury,AR,3.045,MTR00041,simulated_smart_meter,2025-08-07T02:28:55.751Z,PLAN004,Time-of-Use,0.2032,null,null,2025-08-07T02:50:58.086Z
CUST00005,Heather,Davis,East Jasonmouth,MH,2.903,MTR00014,simulated_smart_meter,2025-08-07T02:28:44.038Z,PLAN005,Weekend Special,0.2211,null,null,2025-08-07T02:50:58.086Z
CUST00006,Diamond,Bennett,Donaldshire,NH,4.027,MTR00018,simulated_smart_meter,2025-08-06T19:54:41.962Z,PLAN001,Fixed Rate 12,0.1418,null,null,2025-08-07T02:50:58.086Z
CUST00007,John,Gallagher,Lake Elizabethshire,FL,2.379,MTR00030,simulated_smart_meter,2025-08-06T19:54:29.937Z,PLAN002,Variable Rate 6,0.1603,null,null,2025-08-07T02:50:58.086Z
CUST00008,Michelle,White,South Tina,KS,1.314,MTR00010,simulated_smart_meter,2025-08-07T02:28:20.587Z,PLAN003,Green Energy 24,0.2267,null,null,2025-08-07T02:50:58.086Z
CUST00009,Melanie,Meyer,Port Anneport,LA,3.809,MTR00028,simulated_smart_meter,2025-08-07T02:28:20.587Z,PLAN004,Time-of-Use,0.2032,null,null,2025-08-07T02:50:58.086Z
CUST00010,April,Burton,Port Matthew,ME,3.437,MTR00032,simulated_smart_meter,2025-08-06T19:53:54.491Z,PLAN005,Weekend Special,0.2211,null,null,2025-08-07T02:50:58.086Z


In [0]:
%sql
select * from `power-catalog`.bronzeschema.bronze_meter_readings

customer_id,kwh_reading,metername,raw_source,reading_timestamp,ingestion_timestamp
CUST00020,4.571,MTR00042,simulated_smart_meter,2025-08-05T22:18:25.377Z,2025-08-05T22:22:10.390Z
CUST00003,3.032,MTR00006,simulated_smart_meter,2025-08-05T22:18:25.377Z,2025-08-05T22:22:10.390Z
CUST00032,0.484,MTR00035,simulated_smart_meter,2025-08-05T22:18:25.377Z,2025-08-05T22:22:10.390Z
CUST00032,3.418,MTR00007,simulated_smart_meter,2025-08-05T22:18:25.377Z,2025-08-05T22:22:10.390Z
CUST00017,3.409,MTR00010,simulated_smart_meter,2025-08-05T22:18:25.377Z,2025-08-05T22:22:10.390Z
CUST00031,4.306,MTR00024,simulated_smart_meter,2025-08-05T22:18:25.377Z,2025-08-05T22:22:10.390Z
CUST00038,3.596,MTR00011,simulated_smart_meter,2025-08-05T22:18:37.353Z,2025-08-05T22:22:10.390Z
CUST00015,4.795,MTR00014,simulated_smart_meter,2025-08-05T22:18:37.353Z,2025-08-05T22:22:10.390Z
CUST00024,1.722,MTR00034,simulated_smart_meter,2025-08-05T22:18:37.353Z,2025-08-05T22:22:10.390Z
CUST00006,2.718,MTR00032,simulated_smart_meter,2025-08-05T22:18:37.353Z,2025-08-05T22:22:10.390Z


In [0]:
%sql

select * from `power-catalog`.silverschema.silver_customers

customer_id,first_name,last_name,email,address,city,state,zip_code,plan_id
CUST00001,Carla,Stewart,carla.stewart@lee.biz,"331 Black Courts, East Alexanderland, ND 86931",Cantuberg,MP,07411,PLAN001
CUST00002,Michael,Mullen,michael.mullen@smith.com,"20764 Morales Mountains Suite 056, Barryport, ID 11663",Riveraville,MA,35752,PLAN002
CUST00003,Christina,Brooks,christina.brooks@abbott-mcguire.com,"04558 Helen Lock Suite 973, Duaneport, ME 04529",Millershire,MS,52368,PLAN003
CUST00004,Jesse,Perez,jesse.perez@barnett.com,"450 Jimenez Land, Melissachester, MN 20073",Suarezbury,AR,86973,PLAN004
CUST00005,Heather,Davis,heather.davis@west.org,"2457 Mendez Fall Suite 208, South Matthew, SC 73631",East Jasonmouth,MH,61053,PLAN005
CUST00006,Diamond,Bennett,diamond.bennett@cruz.info,"87598 Natalie Drives Suite 307, Lake Tasha, PW 89839",Donaldshire,NH,97398,PLAN001
CUST00007,John,Gallagher,john.gallagher@owens.com,"25845 Alexander Roads, Jonesshire, UT 66358",Lake Elizabethshire,FL,03137,PLAN002
CUST00008,Michelle,White,michelle.white@bennett.biz,"6827 Gerald Pass Suite 798, Port Lesliechester, FM 29552",South Tina,KS,22080,PLAN003
CUST00009,Melanie,Meyer,melanie.meyer@hart-barton.com,"0376 Harper Neck, Jonesland, IA 48760",Port Anneport,LA,38661,PLAN004
CUST00010,April,Burton,april.burton@duncan.com,"817 Brown Pike Apt. 228, Port Todd, VI 79327",Port Matthew,ME,42957,PLAN005


In [0]:
%sql
select * from `power-catalog`.bronzeschema.bronze_customers

customer_id,first_name,last_name,email,address,city,state,zip_code,plan_id
CUST00001,Carla,Stewart,carla.stewart@lee.biz,"331 Black Courts, East Alexanderland, ND 86931",Cantuberg,MP,07411,PLAN001
CUST00002,Michael,Mullen,michael.mullen@smith.com,"20764 Morales Mountains Suite 056, Barryport, ID 11663",Riveraville,MA,35752,PLAN002
CUST00003,Christina,Brooks,christina.brooks@abbott-mcguire.com,"04558 Helen Lock Suite 973, Duaneport, ME 04529",Millershire,MS,52368,PLAN003
CUST00004,Jesse,Perez,jesse.perez@barnett.com,"450 Jimenez Land, Melissachester, MN 20073",Suarezbury,AR,86973,PLAN004
CUST00005,Heather,Davis,heather.davis@west.org,"2457 Mendez Fall Suite 208, South Matthew, SC 73631",East Jasonmouth,MH,61053,PLAN005
CUST00006,Diamond,Bennett,diamond.bennett@cruz.info,"87598 Natalie Drives Suite 307, Lake Tasha, PW 89839",Donaldshire,NH,97398,PLAN001
CUST00007,John,Gallagher,john.gallagher@owens.com,"25845 Alexander Roads, Jonesshire, UT 66358",Lake Elizabethshire,FL,03137,PLAN002
CUST00008,Michelle,White,michelle.white@bennett.biz,"6827 Gerald Pass Suite 798, Port Lesliechester, FM 29552",South Tina,KS,22080,PLAN003
CUST00009,Melanie,Meyer,melanie.meyer@hart-barton.com,"0376 Harper Neck, Jonesland, IA 48760",Port Anneport,LA,38661,PLAN004
CUST00010,April,Burton,april.burton@duncan.com,"817 Brown Pike Apt. 228, Port Todd, VI 79327",Port Matthew,ME,42957,PLAN005


In [0]:
%sql
select * from `power-catalog`.goldschema.gold_daily_customer_kwh_summary



customer_id,first_name,last_name,city,state,plan_id,plan_name,rate_per_kwh,reading_date,total_kwh_daily,num_readings_daily,avg_kwh_per_reading_daily,calculated_cost_daily
CUST00033,Tabitha,Wallace,Curtiston,UT,PLAN003,Green Energy 24,0.2267,2025-08-05,0.12,1,0.12,0.03
CUST00003,Christina,Brooks,Millershire,MS,PLAN003,Green Energy 24,0.2267,2025-08-05,7.28,2,3.64,1.65
CUST00006,Diamond,Bennett,Donaldshire,NH,PLAN001,Fixed Rate 12,0.1418,2025-08-05,2.72,1,2.72,0.39
CUST00007,John,Gallagher,Lake Elizabethshire,FL,PLAN002,Variable Rate 6,0.1603,2025-08-05,4.38,1,4.38,0.7
CUST00015,Wanda,Barker,Lake Kristen,LA,PLAN005,Weekend Special,0.2211,2025-08-05,7.28,2,3.64,1.61
CUST00016,Angela,Baker,New Hunter,NH,PLAN001,Fixed Rate 12,0.1418,2025-08-05,1.73,1,1.73,0.25
CUST00017,Diana,Campbell,Port Christopher,LA,PLAN002,Variable Rate 6,0.1603,2025-08-05,3.41,1,3.41,0.55
CUST00020,Cameron,Morrison,Port Denise,WI,PLAN005,Weekend Special,0.2211,2025-08-05,4.57,1,4.57,1.01
CUST00022,John,Peterson,Lisaville,FL,PLAN002,Variable Rate 6,0.1603,2025-08-05,2.82,1,2.82,0.45
CUST00024,Julie,White,South Samuelland,WI,PLAN004,Time-of-Use,0.2032,2025-08-05,1.72,1,1.72,0.35


In [0]:
%sql
DESCRIBE TABLE EXTENDED `power-catalog`.goldschema.gold_daily_plan_kwh_summary

col_name,data_type,comment
plan_id,string,null
plan_name,string,null
rate_per_kwh,double,null
reading_date,date,null
total_kwh_daily_plan,double,null
num_readings_daily_plan,bigint,null
avg_kwh_per_reading_daily_plan,double,null
calculated_cost_daily_plan,double,null
,,
# Detailed Table Information,,


In [0]:
%sql
select * from `power-catalog`.goldschema.gold_daily_customer_kwh_summary order by customer_id

customer_id,first_name,last_name,city,state,plan_id,plan_name,rate_per_kwh,reading_date,total_kwh_daily,num_readings_daily,avg_kwh_per_reading_daily,calculated_cost_daily
CUST00001,Carla,Stewart,Cantuberg,MP,PLAN001,Fixed Rate 12,0.1418,2025-08-07,13.49,4,3.37,1.91
CUST00001,Carla,Stewart,Cantuberg,MP,PLAN001,Fixed Rate 12,0.1418,2025-08-06,11.64,7,1.66,1.65
CUST00002,Michael,Mullen,Riveraville,MA,PLAN002,Variable Rate 6,0.1603,2025-08-07,6.93,2,3.47,1.11
CUST00002,Michael,Mullen,Riveraville,MA,PLAN002,Variable Rate 6,0.1603,2025-08-06,3.11,2,1.55,0.5
CUST00003,Christina,Brooks,Millershire,MS,PLAN003,Green Energy 24,0.2267,2025-08-06,7.24,3,2.41,1.64
CUST00003,Christina,Brooks,Millershire,MS,PLAN003,Green Energy 24,0.2267,2025-08-07,9.09,3,3.03,2.06
CUST00003,Christina,Brooks,Millershire,MS,PLAN003,Green Energy 24,0.2267,2025-08-05,7.28,2,3.64,1.65
CUST00004,Jesse,Perez,Suarezbury,AR,PLAN004,Time-of-Use,0.2032,2025-08-06,1.38,1,1.38,0.28
CUST00004,Jesse,Perez,Suarezbury,AR,PLAN004,Time-of-Use,0.2032,2025-08-07,4.44,2,2.22,0.9
CUST00005,Heather,Davis,East Jasonmouth,MH,PLAN005,Weekend Special,0.2211,2025-08-06,7.3,3,2.43,1.61


In [0]:
%sql
drop table `power-catalog`.goldschema.gold_daily_plan_kwh_summary